In [1]:
"""NOTES: Batch data is different each time in keras, which result in slight differences in results."""
import pickle
import keras
import numpy as np
import pandas as pd
import os
from keras.layers import Dropout, MaxPooling1D, Reshape, multiply, Conv1D, GlobalAveragePooling1D, Dense,LSTM
# from keras.models import Input, Model, load_model
from keras.models import  Model, load_model
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, LearningRateScheduler , EarlyStopping
from scipy.interpolate import splev, splrep
from sklearn.metrics import confusion_matrix, f1_score
import random


In [2]:
base_dir = "./apnea-ecg-database-1.0.0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
ir = 3  # interpolate interval
before = 2
after = 2
# normalize
scaler = lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr))


In [3]:
def load_data(path):
    tm = np.arange(0, (before + 1 + after) * 60, step=1 / float(ir))
    with open(os.path.join(base_dir, path), 'rb') as f:  # read preprocessing result
        apnea_ecg = pickle.load(f)
    x_train1  = [] 
    
    o_train, y_train = apnea_ecg["o_train"], apnea_ecg["y_train"]
    groups_train = apnea_ecg["groups_train"]
    
    #print("len(o_train): ",len(o_train))
    
    for i in range(len(o_train)):
        (rri_tm, rri_signal), (ampl_tm, ampl_siganl) = o_train[i]
        # Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        ampl_interp_signal = splev(tm, splrep(ampl_tm, scaler(ampl_siganl), k=3), ext=1)
        x_train1.append([rri_interp_signal, ampl_interp_signal])  # 5-minute-long segment
       
        
    x_training1,y_training,groups_training = [],[],[]
    x_val1,y_val,groups_val = [],[],[]

    # trainlist = random.sample(range(len(o_train)),int(len(o_train)*0.7))    
    with open(os.path.join('./train_list_data', 'trainlist.pkl'), 'rb') as f:  # read preprocessing result
        
        trainlist = pickle.load(f)

    #random.shuffle(trainlist)

    num = [i for i in range(len(o_train))]
    #trainlist = num[100:]
    vallist = set(num) - set(trainlist)
    vallist = list(vallist)
    for i in trainlist:
        x_training1.append(x_train1[i])
        y_training.append(y_train[i])
        groups_training.append(groups_train[i])
        
    for i in vallist:
        x_val1.append(x_train1[i])
        y_val.append(y_train[i])
        groups_val.append(groups_train[i])

    #print('y_val: ', y_val)
    
    x_training1 = np.array(x_training1, dtype="float32").transpose((0, 2, 1))
    y_training = np.array(y_training, dtype="float32")
    x_val1 = np.array(x_val1, dtype="float32").transpose((0, 2, 1))
    y_val = np.array(y_val, dtype="float32")
    x_test1 = []
    o_test, y_test = apnea_ecg["o_test"], apnea_ecg["y_test"]
    groups_test = apnea_ecg["groups_test"]

    for i in range(len(o_test)):
        (rri_tm, rri_signal), (ampl_tm, ampl_siganl) = o_test[i]
        # Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        ampl_interp_signal = splev(tm, splrep(ampl_tm, scaler(ampl_siganl), k=3), ext=1)
        x_test1.append([rri_interp_signal, ampl_interp_signal])
    x_test1 = np.array(x_test1, dtype="float32").transpose((0, 2, 1))
    y_test = np.array(y_test, dtype="float32")

    return x_training1, y_training, groups_training, x_val1, y_val, groups_val, x_test1, y_test, groups_test


In [4]:
def lr_schedule(epoch, lr):
    if epoch > 30 and (epoch - 1) % 10 == 0:
        lr *= 0.1
    #print("Learning rate: ", lr)
    return lr



In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, GlobalAveragePooling1D, Dense, Reshape, multiply, Input, Dropout, LayerNormalization, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

def multi_head_self_attention(x, num_heads=4, key_dim=32):
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(x, x)
    attn_output = Dropout(0.1)(attn_output)
    out = LayerNormalization(epsilon=1e-6)(x + attn_output)
    return out

def create_model(input_a_shape, weight=1e-3):
    input1 = Input(shape=input_a_shape)
    x1 = Conv1D(16, kernel_size=11, strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight))(input1)
    x1 = Conv1D(24, kernel_size=11, strides=2, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight))(x1)
    x1 = MaxPooling1D(pool_size=3, padding="same")(x1)
    x1 = Conv1D(32, kernel_size=11, strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight))(x1)
    x1 = MaxPooling1D(pool_size=5, padding="same")(x1)
    
    x2 = Conv1D(16, kernel_size=15, strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight))(input1)
    x2 = Conv1D(24, kernel_size=15, strides=2, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight))(x2)
    x2 = MaxPooling1D(pool_size=3, padding="same")(x2)
    x2 = Conv1D(32, kernel_size=15, strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight))(x2)
    x2 = MaxPooling1D(pool_size=5, padding="same")(x2)
    
    concat = keras.layers.concatenate([x1, x2], name="Concat_Layer", axis=-1)
    
    # LAFM
    concat = LSTM(64, return_sequences=True)(concat)
    concat = multi_head_self_attention(concat, num_heads=4, key_dim=32)

    # Channel Attention
    squeeze = GlobalAveragePooling1D()(concat)
    excitation = Dense(32, activation='relu')(squeeze)
    excitation = Dense(64, activation='sigmoid')(excitation)
    excitation = Reshape((1, 64))(excitation)
    scale = multiply([concat, excitation])
    
    scale = LSTM(32, return_sequences=True)(scale)
    scale = multi_head_self_attention(scale, num_heads=4, key_dim=32)  # Second MHSA Layer
    
    # Channel Attention
    squeeze = GlobalAveragePooling1D()(scale)
    excitation = Dense(16, activation='relu')(squeeze)
    excitation = Dense(32, activation='sigmoid')(excitation)
    excitation = Reshape((1, 32))(excitation)
    scale = multiply([scale, excitation])
    
    scale = LSTM(32, return_sequences=True)(scale)
    scale = multi_head_self_attention(scale, num_heads=4, key_dim=32)  # Third MHSA Layer
    
    x = GlobalAveragePooling1D()(scale)
    outputs = Dense(2, activation='softmax', name="Output_Layer")(x)
    
    model = Model(inputs=[input1], outputs=outputs)
    return model


In [6]:
import time

path = "apnea-ecg.pkl"
x_train1, y_train, groups_train, x_val1, y_val, groups_val, x_test1, y_test, groups_test = load_data(path)    
y_train = keras.utils.to_categorical(y_train, num_classes=2)  # Convert to two categories
y_val = keras.utils.to_categorical(y_val, num_classes=2)
y_test = keras.utils.to_categorical(y_test, num_classes=2)
#print('input_shape', x_train1.shape)


In [7]:
print('x_train1 length: ', len(x_train1))
print('x_val1 length: ', len(x_val1))
print('x_test1 length: ', len(x_test1))


x_train1 length:  11696
x_val1 length:  5013
x_test1 length:  16945


In [7]:

# training
#me
callback_EarlyStopp = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Record the start time
start_time = time.time()

BestAcc = 0
N = 10
Result = np.zeros(N)

for i in range(N):
    model = create_model(x_train1.shape[1:])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    if i==0:
        model.summary()
    
    print(f'i = {i}')
    
    filepath = 'weights.best.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
    lr_scheduler = LearningRateScheduler(lr_schedule)
    # me  +callback
    #callbacks_list = [lr_scheduler, checkpoint]
    callbacks_list = [lr_scheduler, checkpoint,callback_EarlyStopp]
    history = model.fit([x_train1], y_train, batch_size=128, epochs=200, verbose=0,
                        validation_data=([x_val1], y_val), callbacks=callbacks_list)


    # test
    #filepath = model_fn
    filepath = './weights.best.hdf5'
    model = load_model(filepath)
    #model = load_model(model_fn)

    loss, accuracy = model.evaluate([x_test1], y_test, verbose=0)
    # save prediction score
    y_score = model.predict([x_test1],  verbose=0)
    output = pd.DataFrame({"y_true": y_test[:, 1], "y_score": y_score[:, 1], "subject": groups_test})
    output.to_csv("./utils/code_for_calculating_per-recording/output/SE-MSCNN.csv", index=False)
    y_true, y_pred = np.argmax(y_test, axis=-1), np.argmax(model.predict([x_test1], batch_size=1024, verbose=0), axis=-1)
    C = confusion_matrix(y_true, y_pred, labels=(1, 0))
    TP, TN, FP, FN = C[0, 0], C[1, 1], C[1, 0], C[0, 1]
    acc, sn, sp = 1. * (TP + TN) / (TP + TN + FP + FN), 1. * TP / (TP + FN), 1. * TN / (TN + FP)
    f1 = f1_score(y_true, y_pred, average='binary')
    #print("i: {}, acc: {}, sn: {}, sp: {}, f1: {}".format(i, acc, sn, sp, f1))
    Result[i] = acc
    # Calculate the elapsed time
    elapsed_time = time.time() - start_time

    # Print the elapsed time
    print(f"Elapsed time: {elapsed_time:.1f} seconds")

    filepath = 'Best_model022.hdf5'
    if BestAcc < acc:
        BestAcc = acc    
        Best_history = history
        Best_i = i
        model.save(filepath)
        print(f'Best model is saved in i = {i} th run with accuracy = {BestAcc}')


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 900, 2)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 900, 16)      368         ['input_1[0][0]']                
                                                                                                  
 conv1d_3 (Conv1D)              (None, 900, 16)      496         ['input_1[0][0]']                
                                                                                                  
 conv1d_6 (Conv1D)              (None, 900, 16)      176         ['input_1[0][0]']                
                                                                                              

In [14]:

filepath = 'Best_model022_2_good.hdf5'
model = load_model(filepath)

loss, accuracy = model.evaluate([x_test1], y_test)
# save prediction score
y_score = model.predict([x_test1])
output = pd.DataFrame({"y_true": y_test[:, 1], "y_score": y_score[:, 1], "subject": groups_test})
output.to_csv("./utils/code_for_calculating_per-recording/output/SE-MSCNN.csv", index=False)
y_true, y_pred = np.argmax(y_test, axis=-1), np.argmax(model.predict([x_test1], batch_size=1024, verbose=1), axis=-1)
C = confusion_matrix(y_true, y_pred, labels=(1, 0))
TP, TN, FP, FN = C[0, 0], C[1, 1], C[1, 0], C[0, 1]
acc, sn, sp = 1. * (TP + TN) / (TP + TN + FP + FN), 1. * TP / (TP + FN), 1. * TN / (TN + FP)
f1 = f1_score(y_true, y_pred, average='binary')
print("acc: {}, sn: {}, sp: {}, f1: {}".format(acc, sn, sp, f1))



17/17 [==============================] - 1s 37ms/step
acc: 0.9147831218648569, sn: 0.8775038520801233, sp: 0.93792443806791, f1: 0.8874863643447094


In [ ]:
    print('Average of all Runs = ', np.average(Result))
    print('Result = ', Result)

    Best_history = history
    
    print("max accuracy = ",max(history.history['accuracy']))    
    print("max val_accuracy = ",max(history.history['val_accuracy']))

    import matplotlib.pyplot as plt
    plt.plot(history.history['accuracy'] ,color='red')
    plt.plot(history.history['val_accuracy'],color='blue')
    plt.show()
    
    plt.plot(history.history['loss'] ,color='red')
    plt.plot(history.history['val_loss'],color='blue')
    plt.show()


In [9]:
#import netron

#filepath = 'Best_model022_2_good_9147.hdf5'
## Open the model in Netron
#netron.start(filepath)